In [42]:
import pandas as pd
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os
from datetime import datetime
import pickle
from src.utils.database import add_to_collection

load_dotenv()
uri = os.getenv('MONGO_URL')

In [2]:
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [3]:
db = client["aus_prop"]

In [43]:
collection_names = [
    "data_suburbs",
    "data_regions",
    "data_states",
    "tables_houses",
    "tables_town_houses",
    "tables_units"
]

In [51]:
with open("TEMP_STORE_AUG.pkl", "rb") as f:
    april = pickle.load(f)

In [52]:
for (collection_name, df) in zip(collection_names, april):
    add_to_collection(db, collection_name, df.reset_index())

In [59]:
collection = db["data_suburbs"]
all_docs = list(collection.find())
df = pd.DataFrame(all_docs)
df.query("name == 'Box Hill'")

,_id,Date,name,vacancy_rate,rental_stock,population,rental_pop,region,state
371,6801df212848fa1da93c01c4,2025-03-01,Box Hill,3.84,340,11391.0,48.8,Whitehorse,Victoria
3132,6801e4a72848fa1da93c5b11,2025-04-01,Box Hill,3.65,324,11391.0,48.8,Whitehorse,Victoria
5817,68a45243e5c12521ee650c85,2025-05-01,Box Hill,2.86,254,11391.0,48.8,Whitehorse,Victoria
8577,68a452eee5c12521ee653953,2025-06-01,Box Hill,2.57,228,11391.0,48.8,Whitehorse,Victoria
11338,68a45319e5c12521ee656622,2025-07-01,Box Hill,2.57,228,11391.0,48.8,Whitehorse,Victoria
14099,68a4533fe5c12521ee6592f1,2025-08-01,Box Hill,2.57,228,11391.0,48.8,Whitehorse,Victoria


In [98]:
df03 = pd.read_csv(os.path.join("data_inv", "202503", "df_data_suburbs.csv"))
df04 = pd.read_csv(os.path.join("data_inv", "202504", "df_data_suburbs.csv"))

In [102]:
date

'202504'

In [104]:
for name in ["df_data_regions.csv", "df_data_states.csv", "df_data_suburbs.csv"]:
    df = pd.read_csv(os.path.join("data_inv", date, name))
    df["Date"] = datetime.strptime(date, "%Y%m")
    df = df[["Date"] + [x for x in df.columns if "Date" not in x]]

    collection = db[name.split("_", 1)[-1].split(".")[0]]
    collection.insert_many(df.to_dict("index").values())

In [97]:
for name in ["data_regions", "data_states", "data_suburbs"]:
    collection = db[name]
    query = {"Date": datetime.strptime("202504", "%Y%m")}
    result = collection.delete_many(query)

In [39]:
collection = db["locations"]

In [43]:
data = df.to_dict("index")